# Generating images with stable diffusion

Learn to generate image with stable diffusion using Huggingface Diffusers

## Outline

1.   Generate an image from a prompt
2.   Generate an image from an image with a prompt


First we install the library.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install -q diffusers transformers accelerate

First we load the model.

In [ ]:
from diffusers import AutoPipelineForText2Image
from IPython.display import clear_output
import torch

pipeline_text2image = AutoPipelineForText2Image.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")

clear_output()

We generate an image from a prompt.

In [ ]:
prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"
negative_prompt = "low quality, bad quality"
image = pipeline_text2image(prompt=prompt, negative_prompt=negative_prompt).images[0]
display(image)

We can also condition the generated image with another image (the previous generated one in this case).

In [ ]:
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import load_image, make_image_grid

# use from_pipe to avoid consuming additional memory when loading a checkpoint
pipeline_image2image = AutoPipelineForImage2Image.from_pipe(pipeline_text2image).to("cuda")

prompt = "An astronaut in mars"
image2image = pipeline_image2image(prompt, negative_prompt=negative_prompt, image=image, strength=0.7, guidance_scale=10.5).images[0]
image_grid = make_image_grid([image, image2image], rows=1, cols=2)
display(image_grid)

We can change how much the original image  affects the output result by modifying the strength parameter

In [ ]:
image2image = pipeline_image2image(prompt, negative_prompt=negative_prompt, image=image, strength=0.4, guidance_scale=10.5).images[0]
image_grid = make_image_grid([image, image2image], rows=1, cols=2)
display(image_grid)

In [ ]:
# @markdown ### Condition the output based on your prompt and image. The image must be square and you first have to upload it:
file_path = "/content/blackhole.jpg" # @param {type:"string"}
new_prompt = "A crusty doughnut with almonds all over it like an flavour explosion" # @param {type:"string"}
image_strength = 0.9 # @param {type:"slider", min:0, max:1, step:0.1}


from PIL import Image
new_image = Image.open(file_path).resize((1024, 1024))

image2image = pipeline_image2image(new_prompt, negative_prompt=negative_prompt, image=new_image, strength=image_strength).images[0]
image_grid = make_image_grid([new_image, image2image], rows=1, cols=2)
display(image_grid)

# Finalizing

When you finish working you have to remember to **stop the runtime**, because there is a time limit and to avoid wasting resources. To stop the runtime click Manage Sessions on the Runtime menu. Once the dialog opens click terminate on the current runtime.

> But when you stop the runtime everything you have not saved is ⚠ **lost** ⚠, so be sure to **download** everything you want to keep before stopping it.


# Credits

Taller Estampa https://tallerestampa.com / https://github.com/estampa

Based on https://huggingface.co/docs/diffusers/en/using-diffusers/sdxl